In [ ]:

%pip install -U langchain-community
%pip install --quiet sentence-transformers
%pip install -U langchain-huggingface
%pip install pypdf

In [3]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA

In [4]:
# from langchain_community.document_loaders import PyPDFDirectoryLoader
# pdf_loader = PyPDFDirectoryLoader("./Data" )
# pages_dir = pdf_loader.load()
# print(len(pages_dir))

In [ ]:
loader = PyPDFLoader("./Ashirvad_Resume.pdf")
documents = loader.load()
print((documents))

In [8]:
#Splitting the data into chunk
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=30, separator="\n")
docs = text_splitter.split_documents(documents=documents)

In [ ]:
#loading the embedding model from huggingface
embedding_model_name = "sentence-transformers/all-MiniLM-L6-v2"
model_kwargs = {"device": "cpu"}
embeddings = HuggingFaceEmbeddings(
  model_name=embedding_model_name,
  model_kwargs=model_kwargs
)

In [35]:
# Create the vector store
vectorstore = FAISS.from_documents(docs, embeddings)

In [37]:
# Save the vector store locally
vectorstore.save_local("faiss_index_")

In [ ]:
# Load Stored Vector
persisted_vectorstore = FAISS.load_local("faiss_index_", embeddings, allow_dangerous_deserialization=True)

In [13]:
#creating a retriever on top of database
retriever = persisted_vectorstore.as_retriever()

In [15]:
from langchain_ollama import OllamaLLM
from langchain.prompts import PromptTemplate

In [17]:
# Initialize the Llama 3.1 model
llm = OllamaLLM(model="llama3.1")

In [19]:
 #Use RetrievalQA chain for orchestration
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)

In [ ]:
query = "Who is Ashirvad Singh?"
qa.run(query)